In [1]:
!pip install kagglehub

     ---------------------------------------- 0.0/40.1 kB ? eta -:--:--
     ------------------- ------------------ 20.5/40.1 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 40.1/40.1 kB 382.4 kB/s eta 0:00:00
   ---------------------------------------- 0.0/70.6 kB ? eta -:--:--
   ---------------------------------------- 70.6/70.6 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/160.4 kB ? eta -:--:--
   -------------------------------------- - 153.6/160.4 kB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 160.4/160.4 kB 4.8 MB/s eta 0:00:00


In [2]:
import kagglehub
import os
import pandas as pd
import numpy as np

dataset_path = kagglehub.dataset_download('saurabhshahane/fake-news-classification')

print("Path to dataset files:", dataset_path)

path_join = os.path.join(dataset_path, 'WELFake_Dataset.csv')
df = pd.read_csv(path_join)

print(df.head())

100%|██████████| 92.1M/92.1M [00:02<00:00, 39.4MB/s]

Extracting files...


Path to dataset files: C:\Users\yuanb\.cache\kagglehub\datasets\saurabhshahane\fake-news-classification\versions\77
   Unnamed: 0                                              title  \
0           0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1           1                                                NaN   
2           2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3           3  Bobby Jindal, raised Hindu, uses story of Chri...   
4           4  SATAN 2: Russia unvelis an image of its terrif...   

                                                text  label  
0  No comment is expected from Barack Obama Membe...      1  
1     Did they post their votes for Hillary already?      1  
2   Now, most of the demonstrators gathered last ...      1  
3  A dozen politically active pastors came here f...      0  
4  The RS-28 Sarmat missile, dubbed Satan 2, will...      1  


In [3]:
sample_1 = df.sample(n=10000)
sample_1

,Unnamed: 0,title,text,label
38285,38285,Fox News Shows REAL REACTION From Demoralized...,The Donald Trump bandwagon is beginning to sho...,1
53990,53990,Democrats push SEC nominees on corporate polit...,(Reuters) - Democrats in the Senate made a con...,0
5454,5454,"Honduran protesters, police clash in growing e...",TEGUCIGALPA (Reuters) - Honduran police fired ...,0
41997,41997,Senate drops proposal to change taxes on start...,SAN FRANCISCO (Reuters) - Venture capitalists ...,0
53056,53056,Kim Jong-un Says North Korea Is Preparing to T...,"SEOUL, South Korea — North Korea’s leader, ...",0
...,...,...,...,...
39029,39029,Xi Jinping May Delay Picking China’s Next Lead...,"BEIJING — The Chinese president, Xi Jinping...",0
6035,6035,Princess Cruise Lines to Pay $40 Million Fine ...,Princess Cruise Lines has pleaded guilty to se...,0
13121,13121,Turkish court frees opposition journalist on b...,ISTANBUL (Reuters) - A Turkish court freed on ...,0
40840,40840,Comment on Democratic Party operative Bob Beck...,New emails show Huma scheming for Hillary \nAs...,1


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [5]:
vectorizer = TfidfVectorizer(
    max_features=2000,
    stop_words='english',
    max_df=0.85,
    min_df=2,
    ngram_range=(1, 2),
    strip_accents='unicode',
    lowercase=True
)

In [6]:
sample_1 = sample_1.dropna()

In [7]:
tfidf_matrix = vectorizer.fit_transform(sample_1['text'])
feature_names = vectorizer.get_feature_names_out()

In [8]:
np.random.choice(feature_names, size=500, replace=False)

array(['sit', 'residents', 'voted', 'defeat', 'attorney general',
       'remove', 'hacking', 'governor', 'germany', 'life', 'refugees',
       'individuals', 'increasingly', 'remained', 'experience',
       'americans', 'assault', 'details', 'nbc', 'named', 'struck',
       'wide', 'worse', 'left', 'love', 'citizen', 'sell', 'information',
       'attacks', 'levels', 'social', 'half', 'reportedly', 'park', '500',
       'homes', 'paul ryan', 'george bush', 'land', 'include', 'truly',
       'la', 'march', 'debt', 'democratic', 'collapse', 'class',
       'affairs', 'telling', 'additional', 'officer', 'political',
       'donors', 'presidential', 'way', 'pro', 'reach', 'fast', 'efforts',
       'ongoing', 'terrorist', 'generation', 'winning', 'discussion',
       'highly', 'ryan', 'video', 'previous', 'floor', 'outcome', 'email',
       'report', 'host', 'form', 'send', 'transition', 'opening',
       'sunday', 'institutions', 'paid', 'professor', 'return',
       'confirmation', 'agre

In [9]:
tfidf_matrix.shape

(9912, 2000)

In [10]:
def find_optimal_clusters(tfidf_matrix, max_clusters=20):
    """
    Standard implementation of elbow method and silhouette score to find optimal number of clusters.
    """
    inertias = []
    silhouette_scores = []
    K_range = range(2, min(max_clusters + 1, (tfidf_matrix.shape)[0]))
    
    print("\nEvaluating optimal number of clusters...")
    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(tfidf_matrix)
        inertias.append(kmeans.inertia_)
        
        # Calculate silhouette score
        score = silhouette_score(tfidf_matrix, kmeans.labels_, sample_size=min(1000, (tfidf_matrix.shape)[0]))
        silhouette_scores.append(score)
        print(f"  k={k}: silhouette={score:.3f}")
    
    # Suggest k with highest silhouette score
    best_k = K_range[np.argmax(silhouette_scores)]
    print(f"\nSuggested number of clusters: {best_k}")
    
    return best_k

In [11]:
(tfidf_matrix.shape)[0]

9912

In [12]:
n_clusters = find_optimal_clusters(tfidf_matrix, max_clusters=20)

kmeans = KMeans(n_clusters=n_clusters, random_state=1, n_init=10)
cluster_labels = kmeans.fit_predict(tfidf_matrix)


Evaluating optimal number of clusters...
  k=2: silhouette=0.012
  k=3: silhouette=0.011
  k=4: silhouette=0.014
  k=5: silhouette=0.015
  k=6: silhouette=0.015
  k=7: silhouette=0.022
  k=8: silhouette=0.017
  k=9: silhouette=0.016
  k=10: silhouette=0.019
  k=11: silhouette=0.018
  k=12: silhouette=0.021
  k=13: silhouette=0.020
  k=14: silhouette=0.019
  k=15: silhouette=0.018
  k=16: silhouette=0.023
  k=17: silhouette=0.022
  k=18: silhouette=0.023
  k=19: silhouette=0.023
  k=20: silhouette=0.020

Suggested number of clusters: 19


In [13]:
cluster_labels
sample_1['cluster'] = cluster_labels

C:\Users\yuanb\AppData\Local\Temp\ipykernel_19296\186062633.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_1['cluster'] = cluster_labels


In [14]:
sample_1[sample_1['cluster'] == 10]

,Unnamed: 0,title,text,label,cluster
69432,69432,Obama Singing ‘Happy Birthday’ To Malia Is Th...,"On July 4, we celebrate our independence each ...",1,10
5734,5734,Trump Is A MONSTROUS Threat To A Vital Govern...,When people talk about the threat that Donald ...,1,10
33256,33256,Fox News FIRES Longtime Executive For Making ...,"For years, Fox News hosts have been spewing ra...",1,10
70862,70862,"‘Why Me?’ Echoed in Ralph Branca’s World, Even...","In 1902, a seamstress named Katherine Berger m...",0,10
49338,49338,BREAKING: Jury Reaches Decision on Oregon Wild...,Pinterest \nThe leaders of the militia who tem...,1,10
...,...,...,...,...,...
22362,22362,Biggest Secret: How Reptilian Human Hybrids Ru...,posted by Eddie “The ultimate measure of a man...,1,10
54108,54108,‘Rogue One’ Reviews: Here’s What the Critics S...,Movie critics got their first look at “Rogue O...,0,10
25586,25586,Once and Always a New Yorker - The New York Times,"I left Manhattan in 1988, the morning after ma...",0,10
60288,60288,"WHAT HAPPENED? In 2014, Quebec tried to pass a...","BNI Store Nov 6 2016 That was then… Sadly, thi...",1,10


In [15]:
for cluster in range(n_clusters):
    cluster_mask = sample_1['cluster'] == cluster
    cluster_data = sample_1[cluster_mask]
    size = len(cluster_data)
    fake_fraction = (cluster_data['label']==0).sum() / size

    print(f'Cluster {cluster}: Size={size}, Fake Fraction={fake_fraction:.2f}')

Cluster 0: Size=1507, Fake Fraction=0.21
Cluster 1: Size=346, Fake Fraction=0.96
Cluster 2: Size=1226, Fake Fraction=0.30
Cluster 3: Size=446, Fake Fraction=0.78
Cluster 4: Size=182, Fake Fraction=0.90
Cluster 5: Size=165, Fake Fraction=0.87
Cluster 6: Size=442, Fake Fraction=0.50
Cluster 7: Size=557, Fake Fraction=0.21
Cluster 8: Size=345, Fake Fraction=0.80
Cluster 9: Size=273, Fake Fraction=0.70
Cluster 10: Size=1301, Fake Fraction=0.31
Cluster 11: Size=509, Fake Fraction=0.67
Cluster 12: Size=198, Fake Fraction=0.75
Cluster 13: Size=1374, Fake Fraction=0.83
Cluster 14: Size=287, Fake Fraction=0.05
Cluster 15: Size=199, Fake Fraction=0.31
Cluster 16: Size=321, Fake Fraction=0.66
Cluster 17: Size=134, Fake Fraction=0.42
Cluster 18: Size=100, Fake Fraction=0.81


In [16]:
# Get top terms per cluster
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names_out()
for cluster in range(n_clusters):
    top_terms = [terms[ind] for ind in order_centroids[cluster, :10]]
    print(f"Cluster {cluster} top terms: {', '.join(top_terms)}")

    

Cluster 0 top terms: news, 2016, facebook, breitbart, video, new, media, said, com, trump
Cluster 1 top terms: mr, mr trump, trump, said, ms, mrs, mrs clinton, new, clinton, president
Cluster 2 top terms: trump, donald, donald trump, said, president, campaign, republican, people, clinton, election
Cluster 3 top terms: party, election, said, percent, vote, government, voters, elections, parliament, minister
Cluster 4 top terms: korea, north korea, north, nuclear, korean, china, missile, said, sanctions, south
Cluster 5 top terms: eu, european, britain, brexit, european union, said, union, minister, brussels, talks
Cluster 6 top terms: police, said, officers, officer, arrested, people, man, killed, shooting, black
Cluster 7 top terms: clinton, hillary, hillary clinton, trump, campaign, emails, email, foundation, state, presidential
Cluster 8 top terms: senate, obamacare, house, ryan, republicans, republican, said, health, legislation, insurance
Cluster 9 top terms: court, supreme court, 

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

cluster_accuracies = {}

for cluster in range(n_clusters):
    
    # Get indices belonging to this cluster
    cluster_indices = np.where(cluster_labels == cluster)[0]
    
    # Extract cluster-specific feature matrix
    X_cluster = tfidf_matrix[cluster_indices]
    
    # Extract labels
    y_cluster = sample_1.iloc[cluster_indices]['label']
    
    # Train/test split
    X_train, X_test, y_train, y_test = train_test_split(
        X_cluster,
        y_cluster,
        test_size=0.2,
        random_state=42,
        stratify=y_cluster
    )
    
    # Train logistic regression
    model = LogisticRegression(
        max_iter=2000,
        C=5,                # increase regularization strength tuning
        solver='liblinear', # often better for smaller datasets
    )

    model.fit(X_train, y_train)
    
    # Predict
    y_pred = model.predict(X_test)
    
    # Evaluate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    cluster_accuracies[cluster] = accuracy
    
    print(f"Cluster {cluster}: Accuracy = {accuracy:.4f}")

Cluster 0: Accuracy = 0.8675
Cluster 1: Accuracy = 0.9571
Cluster 2: Accuracy = 0.9024
Cluster 3: Accuracy = 0.8667
Cluster 4: Accuracy = 0.8919
Cluster 5: Accuracy = 0.9394
Cluster 6: Accuracy = 0.9101
Cluster 7: Accuracy = 0.8929
Cluster 8: Accuracy = 0.8551
Cluster 9: Accuracy = 0.8182
Cluster 10: Accuracy = 0.8161
Cluster 11: Accuracy = 0.8922
Cluster 12: Accuracy = 0.8250
Cluster 13: Accuracy = 0.8873
Cluster 14: Accuracy = 0.9483
Cluster 15: Accuracy = 0.9000
Cluster 16: Accuracy = 0.8769
Cluster 17: Accuracy = 0.8519
Cluster 18: Accuracy = 0.8000
